In [2]:
! pip install biopython

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB 435.7 kB/s eta 0:00:07
   - -------------------------------------- 0.1/2.7 MB 558.5 kB/s eta 0:00:05
   - -------------------------------------- 0.1/2.7 MB 717.5 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/2.7 MB 953.7 kB/s eta 0:00:03
   --- ------------------------------------ 0.2/2.7 MB 953.7 kB/s eta 0:00:03
   --- ------------------------------------ 0.2/2.7 MB 724.0 kB/s eta 0:00:04
   --- ------------------------------------ 0.3/2.7 MB 714.4 kB/s eta 0:00:04
   ---- ----------------------------------- 0.3/2.7 MB 707.1 kB/s eta 0:00:04
   ----- ---------------------------------- 0.4/2.7 MB 865.0 kB/s eta 0:00:03
   ------ --------------------------------- 0.4/2.7 MB 865.6 kB/s eta 0:00:03
   ------- -------------------------------- 0.5/2.7 MB 981.7 kB/s eta 0:00:03
   -

In [3]:
! pip install biopython

from Bio import SeqIO
from Bio.Align import PairwiseAligner
aligner = PairwiseAligner ()
aligner.mode = 'global'
aligner.match_score = 5
aligner.mismatch_score = -4
aligner.open_gap_score = -10
aligner.extend_gap_score = -0.5
seqs = {}
for record in SeqIO.parse("SARS_Spike_NSP12.fasta", "fasta"):
    seqs[record.id] = record.seq

print('Выравнивание NSP12')
for align in aligner.align(seqs['SARS-CoV_NSP12'], seqs['SARS-CoV-2_NSP12']):
    print(align)

print('Выравнивание Spike')
print(aligner.align(seqs['SARS-CoV_Spike'], seqs['SARS-CoV-2_Spike'])[0])

Выравнивание NSP12
target            0 SADASTFLNRVCGVSAARLTPCGTGTSTDVVYRAFDIYNEKVAGFAKFLKTNCCRFQEKD
                  0 ||||..|||||||||||||||||||||||||||||||||.||||||||||||||||||||
query             0 SADAQSFLNRVCGVSAARLTPCGTGTSTDVVYRAFDIYNDKVAGFAKFLKTNCCRFQEKD

target           60 EEGNLLDSYFVVKRHTMSNYQHEETIYNLVKDCPAVAVHDFFKFRVDGDMVPHISRQRLT
                 60 |..||.||||||||||.||||||||||||.|||||||.|||||||.||||||||||||||
query            60 EDDNLIDSYFVVKRHTFSNYQHEETIYNLLKDCPAVAKHDFFKFRIDGDMVPHISRQRLT

target          120 KYTMADLVYALRHFDEGNCDTLKEILVTYNCCDDDYFNKKDWYDFVENPDILRVYANLGE
                120 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
query           120 KYTMADLVYALRHFDEGNCDTLKEILVTYNCCDDDYFNKKDWYDFVENPDILRVYANLGE

target          180 RVRQSLLKTVQFCDAMRDAGIVGVLTLDNQDLNGNWYDFGDFVQVAPGCGVPIVDSYYSL
                180 ||||.||||||||||||.||||||||||||||||||||||||.|..||.|||.|||||||
query           180 RVRQALLKTVQFCDAMRNAGIVGVLTLDNQDLNGNWYDFGDFIQTTPGSGVPVVDSYYSL

targe

In [ ]:
from Bio import AlignIO
aln = AlignIO.read("SARS_CoV_2_Russia_aligned.fasta", "fasta")

#Функция, настраивающая координаты гена с учетом гэпов
def coord(aln, local):
    seq = aln[0]
    _global_ = -1
    for j in range(len(seq)):
        if aln[0][j] != '-':
            _global_ += 1
        if _global_ == local:
            return j

    
from SARS_CoV_2_genes import gene_coordinates
start = coord(aln, gene_coordinates ["Spike"][0] - 1)#обозначаем начало гена
end = coord(aln, gene_coordinates ["Spike"] [1])#обозначаем конец гена
reference = aln[0, start:end].seq #обозначаем участок референсного штамма, который будем сравнивать
n = 0 #счетчик мутаций

for i in range(1,len (aln)):
    new = aln[i, start:end].seq
    for j in range(len(new)):
        if new[j] != reference[j]: #если нуклеотид несовпадает с референсным, увеличиваем переменную k
            n += 1
            
count_Spike = n/((end-start)*len (aln))
print ('Среднее число мутаций в гене Spike', count_Spike)


start = coord(aln, gene_coordinates ["NSP12"][0] - 1)
end = coord(aln, gene_coordinates ["NSP12"] [1])
reference = aln[0, start:end].seq
n = 0
for i in range(1,len (aln)):
    new = aln[i, start:end].seq
    for j in range(len (new)):
        if new[j] != reference[j]:
            n += 1
            
count_NSP12 = n/((end-start)*len (aln))
print ('Среднее число мутаций в гене NSP12', count_NSP12)


'Полученные значения указывают на то, что мутации в гене Spike происходят в 3 раза чаще'
'Это объясняется тем, что гены полимеразы (NSP12) - более консервативны, чем гены Spike-белков'
'Spike-белки образуют выросты на поверхности вириона и довольно часто мутируют, чтобы обходить имунную систему пораженного организма'

In [ ]:
from Bio.Align import PairwiseAligner, substitution_matrices
from Bio import SeqIO
import numpy as np

aligner = PairwiseAligner()
aligner.mode = 'global'
aligner.substitution_matrix = substitution_matrices.load('NUC.4.4')
aligner.open_gap_score = -10
aligner.extend_gap_score = -0.5

seqs = SeqIO.parse('all.fasta', 'fasta')
sequences = []
for seq in seqs:
    sequences.append(seq)
names = ['SARS-CoV-2', 'SARS-CoV', 'MERS-CoV', 'HCoV-OC43', 'HCoV-NL63', 'HCoV-HKU1', 'HCoV-229E']
matrix = [[] for i in range(8)]
matrix[0].append(0)
for i in range(1, 8):
    matrix[i].append(names[i - 1])
    matrix[0].append(names[i - 1])
for j in range(7):
    for k in range(7):
        score = aligner.score(sequences[j], sequences[k])
        matrix[j + 1].append(score)
for i in range(8):
    print(*matrix[i], sep='\t\t')

scores = []
for i in range(1, 8):
    for j in range(i + 1, 8):
        scores.append(matrix[i][j])
maximum = min(scores)
minimum = max(scores)
for i in range(1, 8):
    for j in range(i + 1, 8):
        if maximum == matrix[i][j]:
            max_pair = [matrix[i][0], matrix[0][j]]
        if minimum == matrix[i][j]:
            min_pair = [matrix[i][0], matrix[0][j]]
print(f'Ближайшие: {", ".join(min_pair)}')
print(f'Далекие: {", ".join(max_pair)}')


In [ ]:
from Bio import SeqIO
from Bio import AlignIO
from Bio import Phylo
from Bio.Phylo.TreeConstruction import DistanceCalculator 
import matplotlib
import matplotlib.pyplot as plt

aln = AlignIO.read('all_aligned.fasta', "fasta")
for i in aln:
    if i.id == 'NC_004718.3':
        i.id = 'SARS-CoV'
    elif i.id == 'NC_002645.1':
        i.id = 'HCoV-229E'
    elif i.id == 'NC_006577.2':
        i.id = 'HCoV-HKU1'
    elif i.id == 'NC_005831.2':
        i.id = 'HCoV-NL63'
    elif i.id == 'KU131570.1':
        i.id = 'HCoV-OC43'
    elif i.id == 'NC_019843.3':
        i.id = 'MERs-CoV'
    elif i.id == 'NC_045512.2':
        i.id = 'SARS-CoV2'
calculator = DistanceCalculator('identity')
distance_matrix = calculator.get_distance(aln)

from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
constructor = DistanceTreeConstructor(calculator)

tree = constructor.build_tree(aln)

fig = Phylo.draw(tree)